In [6]:
import pandas as pd
import numpy as np

from scipy.stats import hypergeom
#import statsmodels.stats.multitest as multi
from helpers import bh

import cobra

import os.path

In [7]:
df_subsystems = pd.read_csv('/Users/zivaskof/Documents/MATLAB/mag/subsystem_enrichment/subsystems_moj.csv')
subsystems = df_subsystems.subsystem.unique()
df_subsystems.head()

,subsystem,reaction
0,"Transport, lysosomal",10FTHF5GLUtl
1,"Transport, mitochondrial",10FTHF5GLUtm
2,"Transport, lysosomal",10FTHF6GLUtl
3,"Transport, mitochondrial",10FTHF6GLUtm
4,"Transport, lysosomal",10FTHF7GLUtl


In [8]:
folder = '/Users/zivaskof/Documents/MATLAB/mag/reaction_dif/tInitReactions'

In [16]:
filenames = os.listdir(folder)
filenames.sort()

for file in filenames:
    if not file.startswith('.'):
        print(file)
        df_reactions = pd.read_csv(os.path.join(folder, file))

        df_enrichment = pd.DataFrame(columns=["subsystem", "p_up", "p_down", "q_up", "q_down", "enrichment", "p_changed", "q_changed", "changed"])
        df_enrichment["subsystem"] = subsystems

        M = len(df_reactions) # number of different reactions in pairs of models
        n_up = sum(df_reactions.enrichment == 1) # number of upregulated reactions in models
        n_down = sum(df_reactions.enrichment == -1)  # number of downregulated reactions in models
        n_changed = sum(df_reactions.changed == 1)  # number of changed reactions in models
        print(n_up, n_down, n_changed)
        for subsystem in subsystems[1:]:
            subsystem_reactions = df_subsystems.loc[df_subsystems.subsystem == subsystem,'reaction'].values
            df_sub = df_reactions[df_reactions['reaction'].isin(subsystem_reactions)]

            #if not take_all:
            # option 1: take only remaining reactions
            N = len(df_sub) # number of reactions in a subsystem
            #else:
            #    # option 2: take all reactions from the original model
            #    N = len(df_subs[df_subs.subsystem == subsystem])
            k_up = sum(df_sub.enrichment == 1)# number of upregulated reactions in a subsystem
            k_down = sum(df_sub.enrichment == -1)# number of downregulated reactions in a subsystem
            k_changed = sum(df_sub.changed == 1)# number of changed reactions in a subsystem

            if n_up:         
                p_up = 1 - hypergeom.cdf(k_up-1, M, n_up, N)                
            else:
                p_up = 1.0

            if n_down:         
                p_down = 1 - hypergeom.cdf(k_down-1, M, n_down, N)                
            else:
                p_down = 1.0

            if n_changed:
                p_changed = 1 - hypergeom.cdf(k_changed, M, n_changed, N)                
            else:
                p_changed = 1

            df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_up'] = p_up
            df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_down'] = p_down
            df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_changed'] = p_changed



        df_enrichment['q_up'] = bh(df_enrichment['p_up'])
        df_enrichment['q_down'] = bh(df_enrichment['p_down'])
        df_enrichment['q_changed'] = bh(df_enrichment['p_changed'])


        df_enrichment.loc[(df_enrichment['q_up']<0.05) & (df_enrichment['q_up']<df_enrichment['q_down']),'enrichment'] = 1
        df_enrichment.loc[(df_enrichment['q_down']<0.05) & (df_enrichment['q_down']<=df_enrichment['q_up']),'enrichment'] = -1
        df_enrichment.loc[(df_enrichment['q_changed']<0.05),'changed'] = 1

        df_test = df_enrichment.loc
        df_enrichment = df_enrichment.fillna(0)
        print(df_enrichment)

        output_path = '/Users/zivaskof/Documents/MATLAB/mag/subsystem_enrichment/tINIT_enrichments'
        extracted_string = file.split("_", maxsplit=3)[-1][:-4]
        new_file_name = 'tINIT_subsystem_enrichment_' + extracted_string + ".csv"
        print(new_file_name)
        csv_file_path = os.path.join(output_path, new_file_name)
        df_enrichment.to_csv(csv_file_path, index=False) 
        print("------------------------------")

tINIT_reaction_diff_KM_001.csv
994 899 1893
                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0     

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1

...

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1

                            subsystem  p_up  p_down      q_up    q_down  \
0                Transport, lysosomal   0.0     0.0  0.000000  0.000000   
1            Transport, mitochondrial   1.0     1.0  1.009091  1.009091   
2    Transport, endoplasmic reticular   1.0     1.0  1.009091  1.009091   
3             Beta-Alanine metabolism   1.0     1.0  1.009091  1.009091   
4                Vitamin D metabolism   1.0     1.0  1.009091  1.009091   
..                                ...   ...     ...       ...       ...   
106               Protein degradation   1.0     1.0  1.009091  1.009091   
107              Protein modification   1.0     1.0  1.009091  1.009091   
108              Vitamin K metabolism   1.0     1.0  1.009091  1.009091   
109                   Drug metabolism   1.0     1.0  1.009091  1.009091   
110                 Protein formation   1.0     1.0  1.009091  1.009091   

     enrichment  p_changed  q_changed  changed  
0             0        0.0        0.0        0  
1